In [1]:
import os
import numpy as np
from tensorflow.keras.applications import Xception

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 13:58:32.434139: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 13:58:32.434176: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 13:58:32.434184: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 13:58:32.434217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 13:58:32.434233: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


83683744/83683744 [==============================] - 32s 0us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 13:59:07.581797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6379 - accuracy: 0.7023

2025-04-02 13:59:15.006662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 10s 395ms/step - loss: 0.6379 - accuracy: 0.7023 - val_loss: 0.5801 - val_accuracy: 0.7349
Epoch 2/100
21/21 [==============================] - 7s 335ms/step - loss: 0.4820 - accuracy: 0.7865 - val_loss: 0.5494 - val_accuracy: 0.7349
Epoch 3/100
21/21 [==============================] - 7s 320ms/step - loss: 0.4188 - accuracy: 0.8301 - val_loss: 0.4296 - val_accuracy: 0.7651
Epoch 4/100
21/21 [==============================] - 7s 326ms/step - loss: 0.3742 - accuracy: 0.8361 - val_loss: 0.3900 - val_accuracy: 0.7831
Epoch 5/100
21/21 [==============================] - 7s 315ms/step - loss: 0.3265 - accuracy: 0.8827 - val_loss: 0.3823 - val_accuracy: 0.7711
Epoch 6/100
21/21 [==============================] - 7s 313ms/step - loss: 0.2830 - accuracy: 0.9038 - val_loss: 0.3551 - val_accuracy: 0.8012
Epoch 7/100
21/21 [==============================] - 7s 310ms/step - loss: 0.2377 - accuracy: 0.9233 - val_loss: 0.3624 - val_accuracy: 0.7952
Epoch 8/10

In [12]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 2s 212ms/step - loss: 0.1760 - accuracy: 0.9398
Validation Accuracy: 93.98%


2025-04-02 14:11:05.272942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 2s 227ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.79      0.75       122
           1       0.19      0.14      0.16        44

    accuracy                           0.61       166
   macro avg       0.45      0.46      0.45       166
weighted avg       0.58      0.61      0.59       166


Confusion Matrix:
[[96 26]
 [38  6]]
